Подключаем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2
from google.colab import files
import io

import statsmodels.stats.api as sms
from math import ceil

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Загружаем файлы с данными и помещаем эти данные в переменные типа датафрейм(или сириес) библиотеки пандас  
data1_1 - эксперимент 1, группа A  
data1_2 - эксперимент 1, группа B  
data2_1 - эксперимент 2, группа A  
data2_2 - эксперимент 2, группа B  

In [15]:
uploaded=files.upload()
data1_1 = pd.read_csv(io.StringIO(uploaded['experiment1_1.csv'].decode('utf-8')),sep=',')
data1_2 = pd.read_csv(io.StringIO(uploaded['experiment1_2.csv'].decode('utf-8')),sep=',')
#data2_1 = pd.read_csv(io.StringIO(uploaded['experiment2_1.csv'].decode('utf-8')),sep=',')
#data2_2 = pd.read_csv(io.StringIO(uploaded['experiment2_2.csv'].decode('utf-8')),sep=',')

Saving experiment1_1.csv to experiment1_1.csv
Saving experiment1_2.csv to experiment1_2.csv


# Эксперимент 1

Очищаем исходные данные от дубликатов

In [16]:
data1_1NoDuplicates = data1_1.drop_duplicates()
data1_2NoDuplicates = data1_2.drop_duplicates()

Считаем количество уникальных сессий для групп A и B

In [17]:
sessionCountA = data1_1NoDuplicates.groupby('session').count().shape[0]
sessionCountB = data1_2NoDuplicates.groupby('session').count().shape[0]
print(sessionCountA, sessionCountB)

40338 40532


Считаем количество конверсий

In [18]:
crACount = data1_1NoDuplicates[data1_1NoDuplicates['event'] == 'payment_done'].shape[0]
crBCount = data1_2NoDuplicates[data1_2NoDuplicates['event'] == 'payment_done'].shape[0]
print(crACount, crBCount)

138 154


Считаем конверсию для групп и смотрим на разность конверсий

In [19]:
crA =  crACount / sessionCountA
crB =  crBCount / sessionCountB
crDelta = crB - crA
print(crA, crB, crDelta)

0.003421091774505429 0.0037994670877331493 0.00037837531322772004


Видим незначительное увеличение конверсии

Теперь проверим, что количество выборки для групп достаточное для достоверного результата:

In [20]:
 # Calculating effect size based on our expected rates
effect_size = sms.proportion_effectsize(crA, crB, method='normal')

# Calculating sample size needed
required_n = sms.NormalIndPower().solve_power(
    effect_size, 
    power=0.8, 
    alpha=0.05, 
    ratio=1
    )

required_n = ceil(required_n)                          # Rounding up to next whole number                          
print(required_n)

394152


Получаем размер выборки - 394152 посещений для каждой группы.  
Это не удовлетваряет нашим группам, где в каждой около 40500 посещений.

Считаем сессии без конверсии

In [21]:
sessionNoCrA = sessionCountA - crACount
sessionNoCrB = sessionCountB - crBCount
print(sessionNoCrA, sessionNoCrB)

40200 40378


Считаем конверсию для объединенной группы A + B

In [22]:
crMean = (crACount + crBCount) / (sessionCountA + sessionCountB)
crMean

0.0036107332756275504

Считаем сколько было бы конверсионных и неконверсионных сессий, если конверсия была бы как у объединенной группы

In [23]:
sessionCrMeanA = sessionCountA * crMean
sessionCrMeanB = sessionCountB * crMean

sessionNoCrMeanA = sessionCountA - sessionCrMeanA
sessionNoCrMeanB = sessionCountB - sessionCrMeanB
print(sessionCrMeanA, sessionCrMeanB, sessionNoCrMeanA, sessionNoCrMeanB)

145.64975887226413 146.35024112773587 40192.35024112774 40385.64975887226


По формуле вычисляем вероятность того, что в группе B было статистически значимое улучшение конверсии и нуль-гипотеза может быть отвергнута

In [24]:
O = np.array([crACount, crBCount, sessionNoCrA, sessionNoCrB])
T = np.array([sessionCrMeanA, sessionCrMeanB, sessionNoCrMeanA, sessionNoCrMeanB])
D = np.sum(np.square(T-O)/T)
pVal = chi2.sf(D, df=1)

print("distance d: {0}\np-value: {1}". format(D, pVal))

distance d: 0.8045371274499714
p-value: 0.3697402956393102


Исходя из полученных данных мы видим, что нуль-гипотеза не может быть отвергнута с вероятностью около 37%

# Эксперимент 2

Очищаем исходные данные от дубликатов

In [35]:
data2_1NoDuplicates = data2_1.drop_duplicates()
data2_2NoDuplicates = data2_2.drop_duplicates()

Считаем количество уникальных сессий для групп A и B

In [27]:
sessionCountA2 = data2_1NoDuplicates.groupby('session').count().shape[0]
sessionCountB2 = data2_2NoDuplicates.groupby('session').count().shape[0]
print(sessionCountA2, sessionCountB2)

40338 40219


Считаем количество конверсий

In [28]:
crA2Count = data2_1NoDuplicates[data2_1NoDuplicates['event'] == 'payment_done'].shape[0]
crB2Count = data2_2NoDuplicates[data2_2NoDuplicates['event'] == 'payment_done'].shape[0]
print(crA2Count, crB2Count)

138 953


Считаем конверсию для групп и смотрим на разность конверсий

In [29]:
crA2 =  crA2Count / sessionCountA2
crB2 =  crB2Count / sessionCountB2
cr2Delta = crB2 - crA2
print(crA2, crB2, cr2Delta)

0.003421091774505429 0.023695268405479995 0.020274176630974566


Видим увеличение конверсии около 2%

Теперь проверим, что количество выборки для групп достаточное для достоверного результата:

In [30]:
 # Calculating effect size based on our expected rates
effect_size = sms.proportion_effectsize(crA2, crB2, method='normal')

# Calculating sample size needed
required_n = sms.NormalIndPower().solve_power(
    effect_size, 
    power=0.8, 
    alpha=0.05, 
    ratio=1
    )

required_n = ceil(required_n)                          # Rounding up to next whole number                          
print(required_n)

426


Получаем размер выборки - 426 посещений для каждой группы.  
Это удовлетваряет нашим группам, где в каждой около 40000 посещений.

Считаем сессии без конверсии

In [31]:
sessionNoCrA2 = sessionCountA2 - crA2Count
sessionNoCrB2 = sessionCountB2 - crB2Count
print(sessionNoCrA2, sessionNoCrB2)

40200 39266


Считаем конверсию для объединенной группы A + B

In [32]:
crMean2 = (crA2Count + crB2Count) / (sessionCountA2 + sessionCountB2)
crMean2

0.013543205432178457

Считаем сколько было бы конверсионных и неконверсионных сессий, если конверсия была бы как у объединенной группы

In [33]:
sessionCrMeanA2 = sessionCountA2 * crMean2
sessionCrMeanB2 = sessionCountB2 * crMean2

sessionNoCrMeanA2 = sessionCountA2 - sessionCrMeanA2
sessionNoCrMeanB2 = sessionCountB2 - sessionCrMeanB2
print(sessionCrMeanA2, sessionCrMeanB2, sessionNoCrMeanA2, sessionNoCrMeanB2)

546.3058207232147 544.6941792767853 39791.69417927678 39674.30582072322


По формуле вычисляем вероятность того, что в группе B было статистически значимое улучшение конверсии и нуль-гипотеза может быть отвергнута

In [34]:
O2 = np.array([crA2Count, crB2Count, sessionNoCrA2, sessionNoCrB2])
T2 = np.array([sessionCrMeanA2, sessionCrMeanB2, sessionNoCrMeanA2, sessionNoCrMeanB2])
D2 = np.sum(np.square(T2-O2)/T2)
pVal2 = chi2.sf(D2, df=1)

print("distance d: {0}\np-value: {1}". format(D2, pVal2))

distance d: 619.6254713603117
p-value: 9.020192551074774e-137


Исходя из полученных данных мы видим, что нуль-гипотеза не может быть отвергнута с вероятностью 902*10^-37%, то есть вероятность того, что тестовая и контрольная группы являются множествами одного распределения крайне мала.  
Также принимая во внимание, что размер выборки нас удовлетваряет и наблюдается прирост конверсии около 2% в абсолютной величине, то можем сделать вывод что можно вводить новую версию сайта в продакшн.